In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from utils.load_dataframe import load_dataframe

pd.set_option('display.max_columns', 50)

In [ ]:
orders = load_dataframe("orders")
items = load_dataframe("order_items")
products = load_dataframe("products")
customers = load_dataframe("customer")

sellers = load_dataframe("sellers")

In [ ]:
full_df = orders.merge(items, on='order_id')
full_df = full_df.merge(products, on='product_id')
full_df = full_df.merge(customers, on='customer_id')
full_df = full_df.merge(sellers, on='seller_id')

# Criar coluna de faturamento para cada pedido
full_df['revenue'] = full_df['price'] * full_df['order_item_id']


In [ ]:
# Faturamento Mensal
full_df['order_month'] = full_df['order_purchase_timestamp'].dt.to_period('M')
revenue_by_month = full_df.groupby('order_month')['revenue'].sum()

# Ticket médio mensal
ticket_by_month = (
    full_df
        .groupby('order_month')
        .agg(ticket=('revenue', 'sum'), pedidos=('order_id', 'nunique'))
)
ticket_by_month['ticket_medio'] = (
    ticket_by_month['ticket'] / ticket_by_month['pedidos']
)

display(revenue_by_month)

# === Gráfico de Faturamento Mensal ===
plt.figure(figsize=(12,6))
plt.plot(
    revenue_by_month.index.to_timestamp(),
    revenue_by_month.values,
    marker='o',
    label='Faturamento',
    color='royalblue'
)
plt.title("Faturamento Mensal")
plt.xlabel("Mês")
plt.ylabel("Faturamento (R$)")
plt.xticks(rotation=45)
plt.grid(True)

# Formata eixo Y (em milhões e milhares)
ax = plt.gca()
ax.yaxis.set_major_formatter(mticker.FuncFormatter(
    lambda x, _: f"R${x/1_000_000:.1f}M" if x >= 1_000_000 else f"R${x/1_000:.0f}K"
))
plt.show()

display(ticket_by_month)

# === Gráfico de Ticket Médio ===
plt.figure(figsize=(12, 6))
plt.plot(
    ticket_by_month.index.to_timestamp(),
    ticket_by_month['ticket_medio'],
    marker='o',
    color='orange'
)
plt.title("Ticket Médio Mensal")
plt.xlabel("Mês")
plt.ylabel("Ticket Médio (R$)")
plt.xticks(rotation=45)
plt.grid(True)

# Formata eixo Y também como moeda
ax = plt.gca()
ax.yaxis.set_major_formatter(mticker.FuncFormatter(
    lambda x, _: f"R${x:,.0f}".replace(",", ".")
))
plt.show()


In [ ]:

# Top 10 categorias por faturamento
top_categories = (
    full_df
        .groupby('product_category_name', observed=True)['revenue']
        .sum()
        .sort_values(ascending=False)
        .head(10)

)

plt.figure(figsize=(10,6))
plt.barh(top_categories.index[::-1], top_categories.values[::-1], color='skyblue')
plt.title("Top 10 Categorias por Faturamento")
plt.xlabel("Faturamento (R$)")
plt.show()

display(top_categories)

# Top 10 vendedores por faturamento
top_sellers = (
    full_df
        .groupby('seller_state')['revenue']
        .sum()
        .sort_values(ascending=False)
        .head(10)
)

display(top_sellers)

plt.figure(figsize=(10,6))
plt.barh(top_sellers.index[::-1], top_sellers.values[::-1], color='lightgreen')
plt.title("Top 10 Sellers por Faturamento")
plt.xlabel("Faturamento (R$)")
plt.show()


# Conclusões: Performance de Vendas

- A Olist apresenta **crescimento contínuo de vendas** ao longo de 2017, com picos no fim do ano.
- O **ticket médio** é estável, mas há potencial para aumento.
- As **categorias mais lucrativas** são de uso cotidiano e tecnologia.
- O **Sudeste concentra** os principais vendedores e oportunidades logísticas.